In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
from pydrake.all import StartMeshcat

# if you're not launching jupyter from the repo root, fix the path:
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

meshcat = StartMeshcat()
print("Meshcat:", meshcat.web_url())

INFO:drake:Meshcat listening for connections at http://localhost:7001


Meshcat: http://localhost:7001


In [2]:
import os
import numpy as np
from pydrake.all import StartMeshcat
from pydrake.multibody.tree import RevoluteJoint
from rocky.build import build_robot_diagram_two
from rocky.ilqr import ILQRController, CostParams
from rocky.dynamics import ArmParams, forward_kinematics

rng = np.random.default_rng()
enemy_config_mode = "random"  # options: "random", "default", "custom"
custom_enemy_degs = [30.0, -100.0]


# --- URDF ---
urdf = "../rocky.urdf"

# --- Build ---
bundle = build_robot_diagram_two(
    urdf_path=urdf,
    time_step=1e-3,
    gravity_vec=(0., 0., 0.),
    meshcat=meshcat,
    disable_link_collisions=True,
)

plant, ctx, sim = bundle.plant, bundle.plant_context, bundle.simulator
ally, enemy = bundle.ally, bundle.enemy

W = plant.world_frame()
enemy_target_frame = plant.GetFrameByName("enemy_target")
ally_target_frame = plant.GetFrameByName("ally_target")
ally_wall_frame = plant.GetFrameByName("wall", model_instance=ally)
ally_glove_frame = plant.GetFrameByName("glove", model_instance=ally)
enemy_wall_frame = plant.GetFrameByName("wall", model_instance=enemy)
enemy_elbow_frame = plant.GetFrameByName("elbow_marker", model_instance=enemy)
enemy_glove_frame = plant.GetFrameByName("glove", model_instance=enemy)


ally_act_idxs = list(plant.GetJointActuatorIndices(ally))
enemy_act_idxs = list(plant.GetJointActuatorIndices(enemy))


# --- Joint helpers ---
def get_revolute_joints_for_instance(plant, instance):
    joints = [plant.get_joint(j) for j in plant.GetJointIndices(instance)]
    return [j for j in joints if isinstance(j, RevoluteJoint)]


def set_angles_for_instance(plant, ctx, instance, angles_rad):
    revs = get_revolute_joints_for_instance(plant, instance)
    for j, q in zip(revs, angles_rad):
        j.set_angle(ctx, float(q))


def set_zero_torques(plant, ctx):
    plant.get_actuation_input_port().FixValue(ctx, np.zeros(plant.num_actuators()))


def get_ally_state(plant, ctx, ally):
    q = plant.GetPositions(ctx, ally)[:2]
    dq = plant.GetVelocities(ctx, ally)[:2]
    return np.hstack([q, dq])


def get_enemy_state(plant, ctx, enemy):
    q = plant.GetPositions(ctx, enemy)[:2]
    dq = plant.GetVelocities(ctx, enemy)[:2]
    return np.hstack([q, dq])


def get_shoulder_in_world(plant, ctx, wall_frame):
    X_W_wall = plant.CalcRelativeTransform(ctx, W, wall_frame)
    shoulder_in_wall = np.array([0.0, 0.0, 1.0])   # joint origin in wall frame
    return X_W_wall.translation() + X_W_wall.rotation().matrix() @ shoulder_in_wall


def express_point_in_wall_2d(plant, ctx, wall_frame, p_W_point, p_W_shoulder):
    X_wall_W = plant.CalcRelativeTransform(ctx, wall_frame, W)
    p_wall_point = X_wall_W.multiply(p_W_point)
    p_wall_shoulder = X_wall_W.multiply(p_W_shoulder)
    rel = p_wall_point - p_wall_shoulder             # [dx, dy, dz] in wall frame
    return np.array([rel[0], rel[2]])                # model x,z  (no sign flip)


def get_ally_shoulder_in_world(plant, ctx, ally_wall_frame):
    return get_shoulder_in_world(plant, ctx, ally_wall_frame)


def get_enemy_shoulder_in_world(plant, ctx, enemy_wall_frame):
    return get_shoulder_in_world(plant, ctx, enemy_wall_frame)


def get_target_pos_2d(plant, ctx):
    p_W_shoulder = get_ally_shoulder_in_world(plant, ctx, ally_wall_frame)
    p_W_T = plant.CalcRelativeTransform(ctx, W, enemy_target_frame).translation()
    return express_point_in_wall_2d(plant, ctx, ally_wall_frame, p_W_T, p_W_shoulder)


def get_enemy_pos_2d(plant, ctx):
    p_W_shoulder = get_ally_shoulder_in_world(plant, ctx, ally_wall_frame)
    p_W_EE = plant.CalcRelativeTransform(ctx, W, enemy_glove_frame).translation()
    return express_point_in_wall_2d(plant, ctx, ally_wall_frame, p_W_EE, p_W_shoulder)


def get_enemy_target_pos_2d_for_enemy(plant, ctx):
    p_W_shoulder = get_enemy_shoulder_in_world(plant, ctx, enemy_wall_frame)
    p_W_T = plant.CalcRelativeTransform(ctx, W, ally_target_frame).translation()
    return express_point_in_wall_2d(plant, ctx, enemy_wall_frame, p_W_T, p_W_shoulder)


def get_ally_pos_2d_for_enemy(plant, ctx):
    p_W_shoulder = get_enemy_shoulder_in_world(plant, ctx, enemy_wall_frame)
    p_W_ally = plant.CalcRelativeTransform(ctx, W, ally_glove_frame).translation()
    return express_point_in_wall_2d(plant, ctx, enemy_wall_frame, p_W_ally, p_W_shoulder)


def get_ally_pos_2d_from_model(x, params):
    return forward_kinematics(x[:2], params)


def get_enemy_pos_2d_from_model(x, params):
    return forward_kinematics(x[:2], params)


def get_enemy_pos_2d_for_enemy(plant, ctx):
    p_W_shoulder = get_enemy_shoulder_in_world(plant, ctx, enemy_wall_frame)
    p_W_enemy = plant.CalcRelativeTransform(ctx, W, enemy_glove_frame).translation()
    return express_point_in_wall_2d(plant, ctx, enemy_wall_frame, p_W_enemy, p_W_shoulder)


def apply_combined_torques(u_ally=None, u_enemy=None):
    tau = np.zeros(plant.num_actuators())
    if u_ally is not None:
        tau[ally_act_idxs[0]] = u_ally[0]
        tau[ally_act_idxs[1]] = u_ally[1]
    if u_enemy is not None:
        tau[enemy_act_idxs[0]] = u_enemy[0]
        tau[enemy_act_idxs[1]] = u_enemy[1]
    plant.get_actuation_input_port().FixValue(ctx, tau)


def apply_ally_torque(plant, ctx, ally, u2):
    apply_combined_torques(u_ally=u2, u_enemy=None)


def apply_enemy_torque(plant, ctx, enemy, u2):
    apply_combined_torques(u_ally=None, u_enemy=u2)


def apply_enemy_torque(plant, ctx, enemy, u2):
    tau = np.zeros(plant.num_actuators())
    idx = list(plant.GetJointActuatorIndices(enemy))
    tau[idx[0]] = u2[0]
    tau[idx[1]] = u2[1]
    plant.get_actuation_input_port().FixValue(ctx, tau)


def randomize_enemy_configuration(
    clearance_buffer=0.02,
    clearance_target=None,
    clearance_ally=None,
    max_tries=200,
):
    """Sample random enemy joint angles that keep the glove away from target/ally."""
    glove_radius = 0.05
    target_radius = 0.03
    clearance_target = clearance_target or (glove_radius + target_radius + clearance_buffer)
    clearance_ally = clearance_ally or (2 * glove_radius + clearance_buffer)

    joints = get_revolute_joints_for_instance(plant, enemy)
    limits = []
    for j in joints:
        lo = j.position_lower_limit()
        hi = j.position_upper_limit()
        if not np.isfinite(lo):
            lo = -np.pi
        if not np.isfinite(hi):
            hi = np.pi
        limits.append((lo, hi))

    for attempt in range(1, max_tries + 1):
        candidate = np.array([rng.uniform(lo, hi) for lo, hi in limits])
        set_angles_for_instance(plant, ctx, enemy, candidate)

        p_enemy = plant.CalcRelativeTransform(ctx, W, enemy_glove_frame).translation()
        p_target = plant.CalcRelativeTransform(ctx, W, enemy_target_frame).translation()
        p_ally = plant.CalcRelativeTransform(ctx, W, ally_glove_frame).translation()
        p_elbow = plant.CalcRelativeTransform(ctx, W, enemy_elbow_frame).translation()
        wall_x = plant.CalcRelativeTransform(ctx, W, enemy_wall_frame).translation()[0]

        if not (p_elbow[0] < wall_x and p_enemy[0] < wall_x):
            continue
        if np.linalg.norm(p_enemy - p_target) < clearance_target:
            continue
        if np.linalg.norm(p_enemy - p_ally) < clearance_ally:
            continue
        print(f"Enemy randomized in {attempt} tries.")
        return candidate

    raise RuntimeError("Failed to sample a collision-free enemy configuration")


params = ArmParams()
cost = CostParams(
    Q=np.diag([2.0, 2.0, 0.1, 0.1]),
    R=0.01 * np.eye(2),
    Qf=np.diag([50.0, 50.0, 1.0, 1.0]),
    target_weight=425.0,
    r_min=0.01, # Turning off Exponential Cost at a certain distance
    avoid_weight=30.0,
    avoid_sigma=0.08,
)

dt_mpc = 0.02  # CHANGED: 0.05 is too large for stable feedback on this arm
horizon = 50   # Adjusted horizon to match new dt (40 * 0.02 = 0.8s lookahead)

ctrl = ILQRController(
    horizon=horizon,
    dt=dt_mpc,
    params=params,
    cost=cost,
    verbose=True,
)

print(ctrl)

# Ally starts from a fixed configuration for repeatability
ally_angles = np.deg2rad([45.0, -130.0])
set_angles_for_instance(plant, ctx, ally, ally_angles)

# Enemy configuration selection
if enemy_config_mode == "random":
    enemy_angles = randomize_enemy_configuration()
elif enemy_config_mode == "default":
    enemy_angles = np.deg2rad([45.0, -140.0])
elif enemy_config_mode == "custom":
    enemy_angles = np.deg2rad(custom_enemy_degs)
else:
    raise ValueError(f"Unknown enemy_config_mode: {enemy_config_mode}")
print("Enemy start angles (deg):", np.rad2deg(enemy_angles))
set_angles_for_instance(plant, ctx, enemy, enemy_angles)

sim.AdvanceTo(0.0)
print("Done.")






ILQRController(
  horizon=50, dt=0.02, max_iter=60, tol=0.001, reg=0.0001, verbose=True
  params: l1=0.5, l2=0.5, m1=0.6, m2=0.5, I1=0.002, I2=0.002, damping=[0.100 0.100], gravity=0, torque_limit=[50.000 50.000]
  cost: Q=[[2.000 0.000 0.000 0.000]
 [0.000 2.000 0.000 0.000]
 [0.000 0.000 0.100 0.000]
 [0.000 0.000 0.000 0.100]], R=[[0.010 0.000]
 [0.000 0.010]], Qf=[[50.000  0.000  0.000  0.000]
 [ 0.000 50.000  0.000  0.000]
 [ 0.000  0.000  1.000  0.000]
 [ 0.000  0.000  0.000  1.000]], target_w=425.0, avoid_w=30.0, avoid_sigma=0.08, x_goal=[0.000 0.000 0.000 0.000]
  alphas=[1.0, 0.5, 0.25, 0.1, 0.05]
)
Enemy randomized in 3 tries.
Enemy start angles (deg): [ -9.0817054 152.9861526]
Done.


In [ ]:
import time

t_final = 10.0
dt_ctrl = ctrl.dt
base_avoid_weight = cost.avoid_weight
avoid_decay_time = 5.0  # seconds to significantly reduce avoidance weight
min_avoid_scale = 0.1

t = sim.get_context().get_time()
u_traj_prev = None
step = 0
mode = "mpc"  # start in MPC mode

print("Starting MPC loop...")

def has_reached_target(ally_pos, target_pos, x,
                       R_target=0.1,
                       R_ally=0.005,
                       vel_tol=0.1):

    # distance between centers in task space
    pos_err = np.linalg.norm(ally_pos - target_pos)

    # geometric contact threshold
    contact_dist = R_target + R_ally

    return pos_err <= contact_dist


while t < t_final:
    # 1) Current state
    x_current = get_ally_state(plant, ctx, ally)

    # 2) Positions
    target_pos_2d = get_target_pos_2d(plant, ctx)
    enemy_pos_2d  = get_enemy_pos_2d(plant, ctx)
    ally_pos_2d   = get_ally_pos_2d_from_model(x_current, params)

    print("target positions: ", target_pos_2d)
    print("enemy positions: ", enemy_pos_2d)
    print("ally positions: ", ally_pos_2d)

    # 3) Check for NaN in state (Simulation exploded)
    if np.any(np.isnan(x_current)):
        print("ERROR: Simulation state is NaN. Physics exploded.")
        break

    # 4) Check target hit and switch to passive mode (one-way)
    if mode == "mpc" and has_reached_target(ally_pos_2d, target_pos_2d, x_current):
        print(f"\n*** Target reached at t={t:.2f}s ***")
        print(f"ally EE pos: {ally_pos_2d}, target: {target_pos_2d}")
        print("*** Target reached; switching to passive mode ***")
        mode = "passive"

    # 5) Control law: MPC vs passive. Can change this to 'attack' / 'defense' next
    if mode == "mpc":
        # Decay enemy-avoidance weight over time
        decay = np.exp(-t / avoid_decay_time)
        avoid_scale = max(min_avoid_scale, decay)
        ctrl.cost.avoid_weight = base_avoid_weight * avoid_scale

        # MPC step
        x_pred, result = ctrl.step_mpc(
            x_current,
            target_pos=target_pos_2d,
            enemy_pos=enemy_pos_2d,
            prev_u_trj=u_traj_prev,
        )

        if np.isfinite(result.cost) and result.converged:
            u_traj_prev = result.u_trj
            u0 = result.u_trj[0]
        else:
            print(f"[Warning] iLQR failed (cost: {result.cost}, conv: {result.converged}). Braking.")
            # Damping torque to stop robot if controller fails
            u0 = params.clip_u(-2.0 * x_current[2:])
            u_traj_prev = None  # Reset warm start
    else:
        # Passive after hit: zero (or damping) torques
        # You can replace np.zeros(2) by a damping law if you want.

        # change here or new mode ==
        u0 = np.zeros(2)


    # 6) Debug prints
    if step % 5 == 0:
        if mode == "mpc":
            dbg_cost = result.cost
            dbg_avoid = ctrl.cost.avoid_weight
        else:
            dbg_cost = float("nan")
            dbg_avoid = ctrl.cost.avoid_weight  # last value from MPC phase

        print(
            f"t={t:.3f} | mode={mode} | "
            f"x={np.array2string(x_current, precision=2)} | "
            f"cost={dbg_cost:.1f} | avoid_w={dbg_avoid:.1f} | "
            f"u0={np.array2string(u0, precision=2)}"
        )

    # 7) Apply control and advance sim
    apply_ally_torque(plant, ctx, ally, u0)
    sim.AdvanceTo(t + dt_ctrl)
    t = sim.get_context().get_time()
    step += 1

print("MPC run complete.")


In [ ]:
# Enemy attack controller (run this cell to let the enemy strike back)
enemy_attack_duration = 8.0

enemy_cost = CostParams(
    Q=np.diag([2.0, 2.0, 0.1, 0.1]),
    R=0.01 * np.eye(2),
    Qf=np.diag([50.0, 50.0, 1.0, 1.0]),
    target_weight=425.0,
    r_min=0.01,
    avoid_weight=30.0,
    avoid_sigma=0.08,
)

enemy_ctrl = ILQRController(
    horizon=50,
    dt=0.02,
    params=params,
    cost=enemy_cost,
    verbose=True,
)

base_enemy_avoid_weight = enemy_cost.avoid_weight
enemy_avoid_decay_time = 5.0
enemy_min_avoid_scale = 0.1
enemy_u_traj_prev = None
enemy_mode = "mpc"

enemy_t_start = sim.get_context().get_time()
enemy_t_final = enemy_t_start + enemy_attack_duration
enemy_t = enemy_t_start
enemy_step = 0
result = None

print("Starting ENEMY MPC loop...")
apply_combined_torques(u_ally=np.zeros(2), u_enemy=np.zeros(2))


def has_enemy_reached_target(enemy_pos, target_pos, x,
                             R_target=0.1,
                             R_enemy=0.005,
                             vel_tol=0.1):
    pos_err = np.linalg.norm(enemy_pos - target_pos)
    contact_dist = R_target + R_enemy
    vel_norm = np.linalg.norm(x[2:])
    return pos_err <= contact_dist and vel_norm < vel_tol


while enemy_t < enemy_t_final:
    x_enemy = get_enemy_state(plant, ctx, enemy)
    enemy_pos_2d = get_enemy_pos_2d_for_enemy(plant, ctx)
    target_pos_2d = get_enemy_target_pos_2d_for_enemy(plant, ctx)
    ally_pos_2d = get_ally_pos_2d_for_enemy(plant, ctx)

    if np.any(np.isnan(x_enemy)):
        print("ERROR: Enemy state is NaN. Stopping enemy controller.")
        break

    if enemy_mode == "mpc" and has_enemy_reached_target(enemy_pos_2d, target_pos_2d, x_enemy):
        print(f"*** Enemy target reached at t={enemy_t:.2f}s ***")
        print(f"enemy EE pos: {enemy_pos_2d}, target: {target_pos_2d}")
        enemy_mode = "passive"
        enemy_u_traj_prev = None

    if enemy_mode == "mpc":
        elapsed = enemy_t - enemy_t_start
        decay = np.exp(-elapsed / enemy_avoid_decay_time)
        avoid_scale = max(enemy_min_avoid_scale, decay)
        enemy_ctrl.cost.avoid_weight = base_enemy_avoid_weight * avoid_scale

        x_pred, result = enemy_ctrl.step_mpc(
            x_enemy,
            target_pos=target_pos_2d,
            enemy_pos=ally_pos_2d,
            prev_u_trj=enemy_u_traj_prev,
        )

        if np.isfinite(result.cost) and result.converged:
            enemy_u_traj_prev = result.u_trj
            u0 = result.u_trj[0]
        else:
            print(f"[Enemy Warning] iLQR failed (cost: {result.cost}, conv: {result.converged}). Braking.")
            u0 = params.clip_u(-2.0 * x_enemy[2:])
            enemy_u_traj_prev = None
    else:
        u0 = np.zeros(2)

    if enemy_step % 5 == 0:
        if enemy_mode == "mpc" and result is not None:
            dbg_cost = result.cost
            dbg_avoid = enemy_ctrl.cost.avoid_weight
        else:
            dbg_cost = float("nan")
            dbg_avoid = enemy_ctrl.cost.avoid_weight

        print(
            f"t={enemy_t:.3f} | mode={enemy_mode} | "
            f"x={np.array2string(x_enemy, precision=2)} | "
            f"cost={dbg_cost:.1f} | avoid_w={dbg_avoid:.1f} | "
            f"u0={np.array2string(u0, precision=2)}"
        )

    apply_combined_torques(u_ally=np.zeros(2), u_enemy=u0)
    sim.AdvanceTo(enemy_t + enemy_ctrl.dt)
    enemy_t = sim.get_context().get_time()
    enemy_step += 1

apply_combined_torques(u_ally=np.zeros(2), u_enemy=np.zeros(2))
print("Enemy MPC run complete.")





In [ ]:
# Combined scenario: ally attacks first, then enemy attacks while ally defends
ally_attack_duration = 6.0
enemy_attack_duration = 8.0

# Controllers for this sequence
ally_attack_ctrl = ILQRController(horizon=50, dt=0.02, params=params, cost=cost, verbose=False)
enemy_ctrl = ILQRController(
    horizon=50,
    dt=0.02,
    params=params,
    cost=CostParams(
        Q=np.diag([2.0, 2.0, 0.1, 0.1]),
        R=0.01 * np.eye(2),
        Qf=np.diag([50.0, 50.0, 1.0, 1.0]),
        target_weight=425.0,
        r_min=0.01,
        avoid_weight=10.0,
        avoid_sigma=0.08,
    ),
    verbose=False,
)
ally_defense_ctrl = ILQRController(
    horizon=50,
    dt=0.02,
    params=params,
    cost=CostParams(
        Q=np.diag([2.0, 2.0, 0.1, 0.1]),
        R=0.01 * np.eye(2),
        Qf=np.diag([50.0, 50.0, 1.0, 1.0]),
        target_weight=500.0,  # chase enemy EE
        r_min=0.0,
        avoid_weight=0.0,
        avoid_sigma=0.0,
    ),
    verbose=False,
)

# Helpers

def has_hit_target(pos, target, R_target=0.1, R_ee=0.005):
    return np.linalg.norm(pos - target) <= (R_target + R_ee)

# Reset recording
meshcat.DeleteRecording()
meshcat.StartRecording()

# Phase 1: ally attacks, enemy passive
ally_u_prev = None
enemy_u_prev = None
ally_mode = "attack"
ally_phase_end = sim.get_context().get_time() + ally_attack_duration

print("Phase 1: Ally attacks, enemy passive")
while sim.get_context().get_time() < ally_phase_end:
    x_ally = get_ally_state(plant, ctx, ally)
    target_pos_2d = get_target_pos_2d(plant, ctx)
    enemy_pos_2d = get_enemy_pos_2d(plant, ctx)

    if np.any(np.isnan(x_ally)):
        print("ERROR: Ally state NaN")
        break

    hit = has_hit_target(get_ally_pos_2d_from_model(x_ally, params), target_pos_2d)
    if hit:
        ally_mode = "hit"
        ally_u = np.zeros(2)
    else:
        x_pred, res = ally_attack_ctrl.step_mpc(
            x_ally,
            target_pos=target_pos_2d,
            enemy_pos=enemy_pos_2d,
            prev_u_trj=ally_u_prev,
        )
        if np.isfinite(res.cost) and res.converged:
            ally_u_prev = res.u_trj
            ally_u = res.u_trj[0]
        else:
            ally_u_prev = None
            ally_u = params.clip_u(-2.0 * x_ally[2:])

    apply_combined_torques(u_ally=ally_u, u_enemy=np.zeros(2))
    sim.AdvanceTo(sim.get_context().get_time() + ally_attack_ctrl.dt)

    if hit:
        break

# Ensure torques zeroed before next phase
apply_combined_torques(u_ally=np.zeros(2), u_enemy=np.zeros(2))

# Phase 2: enemy attacks, ally defends by blocking enemy EE
print("Phase 2: Enemy attacks, ally defends")

enemy_u_prev = None
ally_def_u_prev = None
enemy_mode = "attack"
phase2_end = sim.get_context().get_time() + enemy_attack_duration

defense_dt = min(enemy_ctrl.dt, ally_defense_ctrl.dt)

while sim.get_context().get_time() < phase2_end:
    # Enemy state and target
    x_enemy = get_enemy_state(plant, ctx, enemy)
    enemy_target_pos = get_enemy_target_pos_2d_for_enemy(plant, ctx)
    ally_pos_for_enemy = get_ally_pos_2d_for_enemy(plant, ctx)

    # Ally state and enemy EE position (blocking target)
    x_ally = get_ally_state(plant, ctx, ally)
    enemy_pos_for_ally = get_enemy_pos_2d(plant, ctx)

    if np.any(np.isnan(x_enemy)) or np.any(np.isnan(x_ally)):
        print("ERROR: NaN state detected; stopping phase 2")
        break

    enemy_hit = has_hit_target(get_enemy_pos_2d_for_enemy(plant, ctx), enemy_target_pos)
    if enemy_hit:
        enemy_mode = "hit"

    # Enemy control
    if enemy_mode == "attack":
        x_pred_e, res_e = enemy_ctrl.step_mpc(
            x_enemy,
            target_pos=enemy_target_pos,
            enemy_pos=ally_pos_for_enemy,
            prev_u_trj=enemy_u_prev,
        )
        if np.isfinite(res_e.cost) and res_e.converged:
            enemy_u_prev = res_e.u_trj
            u_enemy = res_e.u_trj[0]
        else:
            enemy_u_prev = None
            u_enemy = params.clip_u(-2.0 * x_enemy[2:])
    else:
        u_enemy = np.zeros(2)

    # Ally defense control (target enemy EE)
    x_pred_a, res_a = ally_defense_ctrl.step_mpc(
        x_ally,
        target_pos=enemy_pos_for_ally,
        enemy_pos=np.zeros(2),  # unused (avoid_weight=0) because enemy isn't obstacle
        prev_u_trj=ally_def_u_prev,
    )
    if np.isfinite(res_a.cost) and res_a.converged:
        ally_def_u_prev = res_a.u_trj
        u_ally = res_a.u_trj[0]
    else:
        ally_def_u_prev = None
        u_ally = params.clip_u(-2.0 * x_ally[2:])

    apply_combined_torques(u_ally=u_ally, u_enemy=u_enemy)
    sim.AdvanceTo(sim.get_context().get_time() + defense_dt)

    if enemy_hit:
        break

# Stop and publish recording
apply_combined_torques(u_ally=np.zeros(2), u_enemy=np.zeros(2))
meshcat.StopRecording()
meshcat.PublishRecording()
print("Combined scenario complete.")



Phase 1: Ally attacks, enemy passive


/Users/omarh/Documents/Repos/MEAM5170/RoboRocky/rocky/ilqr.py:135: RuntimeWarning: overflow encountered in matmul
  l += 0.5 * dx.T @ cost.Q @ dx + 0.5 * u.T @ cost.R @ u
/Users/omarh/Documents/Repos/MEAM5170/RoboRocky/rocky/dynamics.py:94: RuntimeWarning: overflow encountered in matmul
  ddq = np.linalg.solve(M, u_clipped - C @ dq - G - damping)


Phase 2: Enemy attacks, ally defends


/Users/omarh/Documents/Repos/MEAM5170/RoboRocky/rocky/ilqr.py:173: RuntimeWarning: overflow encountered in matmul
  l   = 0.5 * (x - cost.x_goal).T @ cost.Qf @ (x - cost.x_goal)


Combined scenario complete.


In [9]:

meshcat.PublishRecording()